# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121647e+02     1.611773e+00
 * time: 0.07040190696716309
     1     1.091033e+01     8.866917e-01
 * time: 1.3839478492736816
     2    -1.192679e+01     9.772462e-01
 * time: 1.4649529457092285
     3    -3.420014e+01     7.344924e-01
 * time: 1.6974709033966064
     4    -4.797862e+01     5.381764e-01
 * time: 1.8075599670410156
     5    -5.687693e+01     2.077723e-01
 * time: 1.9158649444580078
     6    -5.973179e+01     1.173230e-01
 * time: 2.001436948776245
     7    -6.082428e+01     6.330651e-02
 * time: 2.3845438957214355
     8    -6.124074e+01     7.871847e-02
 * time: 2.45560884475708
     9    -6.156129e+01     3.617455e-02
 * time: 2.5272388458251953
    10    -6.177464e+01     2.774036e-02
 * time: 2.6046860218048096
    11    -6.194450e+01     2.400305e-02
 * time: 2.6761720180511475
    12    -6.202145e+01     2.203765e-02
 * time: 2.7607948780059814
    13    -6.210178e+01     1.556112e-02
 * time: 2.831323862075

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671076
    AtomicLocal         -18.8557701
    AtomicNonlocal      14.8522655
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485381 
    Xc                  -19.3336824

    total               -62.261666459017
